In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import psycopg2
import pandas as pd

In [5]:
DBNAME = "opportunity_youth"

In [6]:
conn = psycopg2.connect(dbname=DBNAME)
cur = conn.cursor()

In [7]:
### checking that all rows are for people, not houses
df = pd.read_sql("SELECT COUNT(*) FROM pums_2017 WHERE rt = 'P' LIMIT 100;", conn)
df

,count
0,359075


In [8]:
### find table names from Database
cur.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cur.fetchall():
    print(table)

('pums_2017',)
('puma_names_2010',)
('wa_jobs_2017',)
('wa_geo_xwalk',)
('ct_puma_xwalk',)


In [9]:
### find King County puma id's from puma_names
df = pd.read_sql("""
SELECT puma, puma_name
FROM puma_names_2010
WHERE state_fips= '53' AND puma_name LIKE '%King%'
LIMIT 20
""",
           conn)

indexer = 0
for i in df['puma_name']:
    print (df['puma'][indexer])
    print (i)
    indexer += 1

11606
King County (Northwest)--Shoreline, Kenmore & Bothell (South) Cities                                                                                                                                                                                                                                                                                                                                                                                                                                                
11607
King County (Northwest)--Redmond, Kirkland Cities, Inglewood & Finn Hill                                                                                                                                                                                                                                                                                                                                                                                                                               

In [10]:
### find seattle puma id's from puma
df = pd.read_sql("""
SELECT puma, puma_name
FROM puma_names_2010
WHERE puma_name LIKE '%Seattle%'
LIMIT 20
""",
           conn)

indexer = 0
for i in df['puma_name']:
    print (df['puma'][indexer])
    print (i)
    indexer += 1

11601
Seattle City (Northwest)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
11602
Seattle City (Northeast)                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [11]:
### limiting data by South King County
pd.read_sql("""
SELECT COUNT(*)
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
""",
            conn)

,count
0,34145


In [31]:
### limiting data by SKC and age from 16 - 24, assigning to ay_df
ay_df = pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 16 AND 24
""",
            conn)

ay_df

,rt,serialno,division,sporder,puma,region,st,adjinc,pwgtp,agep,...,pwgtp71,pwgtp72,pwgtp73,pwgtp74,pwgtp75,pwgtp76,pwgtp77,pwgtp78,pwgtp79,pwgtp80
0,P,2013000102767,9,03,11615,4,53,1061971,17.0,22.0,...,27.0,18.0,18.0,16.0,26.0,5.0,5.0,5.0,4.0,17.0
1,P,2013000103566,9,03,11611,4,53,1061971,20.0,17.0,...,37.0,18.0,32.0,4.0,17.0,28.0,40.0,18.0,21.0,19.0
2,P,2013000105303,9,02,11615,4,53,1061971,17.0,23.0,...,28.0,16.0,6.0,5.0,4.0,16.0,28.0,5.0,17.0,17.0
3,P,2013000106284,9,01,11614,4,53,1061971,4.0,23.0,...,4.0,5.0,4.0,6.0,8.0,5.0,5.0,1.0,1.0,3.0
4,P,2013000106963,9,02,11612,4,53,1061971,43.0,19.0,...,15.0,50.0,43.0,41.0,15.0,28.0,41.0,57.0,53.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3477,P,2013000088597,9,03,11610,4,53,1061971,21.0,17.0,...,19.0,22.0,17.0,24.0,7.0,7.0,7.0,23.0,40.0,39.0
3478,P,2013000075342,9,04,11611,4,53,1061971,34.0,22.0,...,54.0,15.0,13.0,32.0,48.0,28.0,65.0,31.0,12.0,52.0
3479,P,2013000088914,9,02,11612,4,53,1061971,36.0,16.0,...,33.0,43.0,69.0,43.0,11.0,63.0,33.0,59.0,38.0,13.0
3480,P,2013000100470,9,03,11613,4,53,1061971,13.0,21.0,...,32.0,3.0,12.0,17.0,12.0,13.0,22.0,19.0,12.0,13.0


In [32]:
### create 3 dataframes based on age grouping of ay_df

ay_df_16_18 =  pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 16 AND 18
""",
            conn)

ay_df_19_21 =  pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 19 AND 21
""",
            conn)

ay_df_22_24 =  pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 22 AND 24
""",
            conn)

In [33]:
ay_df_16_18

,rt,serialno,division,sporder,puma,region,st,adjinc,pwgtp,agep,...,pwgtp71,pwgtp72,pwgtp73,pwgtp74,pwgtp75,pwgtp76,pwgtp77,pwgtp78,pwgtp79,pwgtp80
0,P,2013000103566,9,03,11611,4,53,1061971,20.0,17.0,...,37.0,18.0,32.0,4.0,17.0,28.0,40.0,18.0,21.0,19.0
1,P,2013000106963,9,03,11612,4,53,1061971,24.0,16.0,...,7.0,25.0,38.0,23.0,7.0,25.0,31.0,19.0,34.0,9.0
2,P,2013000115983,9,03,11614,4,53,1061971,4.0,16.0,...,4.0,2.0,4.0,5.0,6.0,8.0,0.0,1.0,7.0,8.0
3,P,2013000121584,9,02,11611,4,53,1061971,19.0,18.0,...,17.0,21.0,32.0,17.0,5.0,12.0,7.0,4.0,30.0,21.0
4,P,2013000121584,9,03,11611,4,53,1061971,18.0,18.0,...,17.0,21.0,32.0,18.0,5.0,11.0,7.0,3.0,30.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,P,2013000068930,9,03,11612,4,53,1061971,13.0,16.0,...,4.0,12.0,13.0,20.0,3.0,11.0,12.0,4.0,12.0,19.0
1309,P,2013000069409,9,04,11614,4,53,1061971,9.0,17.0,...,3.0,6.0,8.0,7.0,2.0,14.0,2.0,9.0,14.0,2.0
1310,P,2013000088914,9,02,11612,4,53,1061971,36.0,16.0,...,33.0,43.0,69.0,43.0,11.0,63.0,33.0,59.0,38.0,13.0
1311,P,2013000088597,9,03,11610,4,53,1061971,21.0,17.0,...,19.0,22.0,17.0,24.0,7.0,7.0,7.0,23.0,40.0,39.0


In [34]:
### limiting data by unemployed status & school status, assigning to oy_df
oy_df = pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 16 AND 24
AND esr != '1'
AND esr != '4'
AND sch = '1'
""",
            conn)

oy_df

,rt,serialno,division,sporder,puma,region,st,adjinc,pwgtp,agep,...,pwgtp71,pwgtp72,pwgtp73,pwgtp74,pwgtp75,pwgtp76,pwgtp77,pwgtp78,pwgtp79,pwgtp80
0,P,2013000106284,9,01,11614,4,53,1061971,4.0,23.0,...,4.0,5.0,4.0,6.0,8.0,5.0,5.0,1.0,1.0,3.0
1,P,2013000111689,9,05,11610,4,53,1061971,40.0,19.0,...,61.0,12.0,51.0,12.0,13.0,41.0,37.0,52.0,32.0,36.0
2,P,2013000118713,9,03,11613,4,53,1061971,25.0,23.0,...,26.0,25.0,29.0,6.0,24.0,26.0,25.0,41.0,24.0,43.0
3,P,2013000121584,9,02,11611,4,53,1061971,19.0,18.0,...,17.0,21.0,32.0,17.0,5.0,12.0,7.0,4.0,30.0,21.0
4,P,2013000128172,9,01,11612,4,53,1061971,118.0,23.0,...,117.0,171.0,181.0,52.0,106.0,143.0,111.0,32.0,94.0,168.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,P,2013000057563,9,05,11611,4,53,1061971,20.0,21.0,...,6.0,7.0,24.0,23.0,6.0,33.0,19.0,33.0,33.0,18.0
406,P,2013000058010,9,02,11614,4,53,1061971,45.0,17.0,...,20.0,56.0,51.0,39.0,40.0,52.0,17.0,48.0,51.0,54.0
407,P,2013000067168,9,03,11615,4,53,1061971,22.0,24.0,...,7.0,20.0,24.0,23.0,41.0,7.0,24.0,24.0,33.0,19.0
408,P,2013000070516,9,03,11612,4,53,1061971,20.0,19.0,...,37.0,20.0,15.0,32.0,6.0,14.0,5.0,6.0,16.0,18.0


In [35]:
### create 3 dataframes based on age grouping of oy_df

oy_df_16_18 = pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 16 AND 18
AND esr != '1'
AND esr != '4'
AND sch = '1'
""",
            conn)

oy_df_19_21 = pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 19 AND 21
AND esr != '1'
AND esr != '4'
AND sch = '1'
""",
            conn)

oy_df_22_24 = pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 22 AND 24
AND esr != '1'
AND esr != '4'
AND sch = '1'
""",
            conn)

In [40]:
print (sum(ay_df_16_18['pwgtp']))
print (sum(oy_df_16_18['pwgtp']))

print (sum(oy_df['pwgtp']))

30141.0
1815.0
11115.0
